In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import keras
from keras.utils import to_categorical
import keras.backend as K
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from keras.layers import TimeDistributed

import torch 
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset

In [24]:
##################################################
### GLOBAL VARIABLES
##################################################
COLUMN_NAMES = [
    'user',
    'activity',
    'timestamp',
    'x-axis',
    'y-axis',
    'z-axis'
]

LABELS = [
    'Downstairs',
    'Jogging',
    'Sitting',
    'Standing',
    'Upstairs',
    'Walking'
]

DATA_PATH = 'data/WISDM_ar_v1.1_raw.txt'
data = pd.read_csv(DATA_PATH, header=None, names=COLUMN_NAMES)
data['z-axis'].replace({';': ''}, regex=True, inplace=True)
data = data.dropna()

In [44]:
RANDOM_SEED = 13

# Data preprocessing
TIME_STEP = 100

# Model
N_CLASSES = 6
N_FEATURES = 3  # x-acceleration, y-acceleration, z-acceleration

# Hyperparameters
N_EPOCHS = 10
LEARNING_RATE = 0.0025

# Hyperparameters optimized
SEGMENT_TIME_SIZE = 180
BATCH_SIZE = 64

In [36]:
# DATA PREPROCESSING
data_convoluted = []
labels = []

# Slide a "SEGMENT_TIME_SIZE" wide window with a step size of "TIME_STEP"
for i in range(0, len(data) - SEGMENT_TIME_SIZE, TIME_STEP):
    x = data['x-axis'].values[i: i + SEGMENT_TIME_SIZE]
    y = data['y-axis'].values[i: i + SEGMENT_TIME_SIZE]
    z = data['z-axis'].values[i: i + SEGMENT_TIME_SIZE]
    data_convoluted.append([x, y, z])

    # Label for a data window is the label that appears most commonly
    label = stats.mode(data['activity'][i: i + SEGMENT_TIME_SIZE])[0][0]
    labels.append(label)

# Convert to numpy
data_convoluted = np.asarray(data_convoluted, dtype=np.float32).transpose(0, 2, 1)

# Integer encoding
label_encoder = LabelEncoder()
labels_int = label_encoder.fit_transform(labels)

# SPLIT INTO TRAINING AND TEST SETS
X_train1, X_test, y_train1, y_test = train_test_split(data_convoluted, labels_int, test_size=0.2, random_state=RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train1, test_size=0.25, random_state=RANDOM_SEED)
print("X train size: ", len(X_train))
print("X test size: ", len(X_test))
print("X val size: ", len(X_val))
print("y train size: ", len(y_train))
print("y test size: ", len(y_test))
print("y val size: ", len(y_val))

X train size:  6588
X test size:  2197
X val size:  2196
y train size:  6588
y test size:  2197
y val size:  2196


In [13]:
y_val[10:80]

array([0, 1, 1, 5, 5, 5, 0, 5, 1, 2, 5, 5, 5, 1, 4, 5, 5, 1, 4, 1, 5, 5,
       5, 0, 4, 1, 1, 5, 5, 5, 1, 1, 2, 1, 1, 3, 5, 5, 5, 5, 1, 5, 1, 1,
       1, 1, 5, 5, 5, 1, 3, 4, 1, 1, 4, 1, 5, 5, 2, 0, 5, 1, 1, 1, 1, 4,
       5, 4, 0, 5], dtype=int64)

In [59]:
np.shape(labels_int)

(10981,)

In [60]:
np.shape(X_train)

(6588, 3, 180)

In [62]:
type(X_train)

list

In [37]:
x_train = X_train.reshape(X_train.shape[0], N_FEATURES, SEGMENT_TIME_SIZE)
x_test = X_test.reshape(X_test.shape[0], N_FEATURES, SEGMENT_TIME_SIZE)
x_val = X_val.reshape(X_val.shape[0], N_FEATURES, SEGMENT_TIME_SIZE)
input_shape = (N_FEATURES, SEGMENT_TIME_SIZE)

In [91]:
x_train.shape

(6588, 3, 180)

In [92]:
x_test.shape

(2197, 3, 180)

In [93]:
type(x_test)

numpy.ndarray

In [79]:
y_train_.shape

(6588,)

In [17]:
type(y_train)

numpy.ndarray

In [27]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2, 3, 4, 5], dtype=int64),
 array([ 591, 2036,  352,  312,  728, 2569], dtype=int64))

In [28]:
np.unique(y_test, return_counts=True)

(array([0, 1, 2, 3, 4, 5], dtype=int64),
 array([212, 676, 140,  78, 258, 833], dtype=int64))

In [29]:
np.unique(y_val, return_counts=True)

(array([0, 1, 2, 3, 4, 5], dtype=int64),
 array([199, 708, 110,  92, 245, 842], dtype=int64))

## pytorch dataset class 

In [38]:
# Define a pytorch dataloader for this dataset
class HAR(Dataset):
    def __init__(self, dfX, dfY):
        self.dfX = dfX
        self.dfY = dfY
        #self.transform = transform

    def __len__(self):
        return len(self.dfX)

    def __getitem__(self, index):
        # Load data and get label
        X = self.dfX[index] 
        y = torch.tensor(int(self.dfY[index]))

        return X, y

In [39]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Define the parameters for the dataloader
params = {'batch_size': BATCH_SIZE,
          'shuffle': True,
          'num_workers': 0}

In [40]:
training_set = HAR(x_train, y_train)
train_loader = DataLoader(training_set, **params)

testing_set = HAR(x_test, y_test)
test_loader = DataLoader(testing_set, **params)

validation_set = HAR(x_val, y_val)
val_loader = DataLoader(validation_set, **params)

In [41]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=N_CLASSES):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 128, kernel_size=8, padding=0)
        self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=5, padding=0)
        self.bn2 = nn.BatchNorm1d(256)
        self.conv3 = nn.Conv1d(256, 128, kernel_size=3, padding=0)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc1 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        #print(np.shape(x))
        out = F.relu(self.bn1(self.conv1(x)))
        #print(np.shape(out))
        out = F.relu(self.bn2(self.conv2(out)))
        #print(np.shape(out))
        out = F.relu(self.bn3(self.conv3(out)))
        #print(np.shape(out))
        out = F.adaptive_avg_pool2d(out, (128,1))
        #print(np.shape(out))
        out = out.view(out.shape[0], -1)
        #print(np.shape(out))
        out = F.relu(self.fc1(out))
        #print(np.shape(out))
        out = F.log_softmax(out, dim=1)
        #print(np.shape(out))
        #print("-------------")
        return out

model = ConvNet(N_CLASSES).to(device)
print(model)

ConvNet(
  (conv1): Conv1d(3, 128, kernel_size=(8,), stride=(1,))
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(128, 256, kernel_size=(5,), stride=(1,))
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(256, 128, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=128, out_features=6, bias=True)
)


In [45]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
#scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=5, cooldown=5)

# Train the model
total_step = len(y_train)
for epoch in range(N_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        #print("Hello")
        images = images.to(device)
        labels = labels.to(device)
        #print("Images shape = ", images.shape)
        #print("Labels shape = ", labels.shape)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch+1, N_EPOCHS, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/6588], Loss: 0.8954
Epoch [2/10], Step [100/6588], Loss: 1.0594
Epoch [3/10], Step [100/6588], Loss: 1.2871
Epoch [4/10], Step [100/6588], Loss: 1.1468
Epoch [5/10], Step [100/6588], Loss: 1.0819
Epoch [6/10], Step [100/6588], Loss: 1.0836
Epoch [7/10], Step [100/6588], Loss: 1.2191
Epoch [8/10], Step [100/6588], Loss: 0.9196
Epoch [9/10], Step [100/6588], Loss: 1.1374
Epoch [10/10], Step [100/6588], Loss: 0.4365


In [46]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        #print("Predicted: ", predicted)
        #print("Labels: ", labels)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy on the test set: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.pth')

Test Accuracy on the test set: 82.84023668639053 %


In [48]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        print("Predicted: ", predicted)
        print("Labels: ", labels)
        print("---------------o0o----------------")
        correct += (predicted == labels).sum().item()

    print('Test Accuracy on the test set: {} %'.format(100 * correct / total))

Predicted:  tensor([3, 5, 5, 0, 2, 5, 1, 1, 5, 1, 5, 1, 5, 1, 5, 5, 1, 5, 0, 5, 3, 1, 5, 1,
        0, 5, 1, 5, 3, 1, 1, 1, 1, 5, 1, 5, 3, 5, 1, 1, 5, 5, 5, 1, 2, 1, 5, 0,
        1, 5, 1, 1, 1, 1, 3, 5, 1, 1, 1, 0, 5, 0, 5, 5])
Labels:  tensor([3, 5, 5, 5, 2, 5, 4, 1, 5, 1, 5, 1, 5, 1, 5, 5, 1, 5, 0, 5, 2, 1, 5, 1,
        0, 1, 1, 4, 3, 4, 1, 1, 1, 5, 1, 5, 3, 4, 1, 1, 4, 5, 5, 1, 2, 1, 5, 0,
        1, 5, 1, 1, 1, 1, 3, 5, 1, 1, 1, 0, 5, 0, 5, 5])
---------------o0o----------------
Predicted:  tensor([5, 1, 1, 5, 5, 5, 1, 5, 1, 0, 1, 0, 1, 1, 1, 5, 5, 1, 5, 1, 5, 5, 1, 1,
        1, 1, 3, 5, 1, 5, 3, 5, 5, 1, 0, 3, 3, 1, 3, 1, 5, 1, 0, 1, 5, 0, 5, 5,
        0, 1, 1, 5, 5, 1, 5, 0, 0, 2, 3, 5, 5, 5, 3, 1])
Labels:  tensor([5, 1, 1, 5, 5, 5, 1, 5, 1, 4, 1, 0, 1, 1, 1, 4, 5, 1, 0, 1, 5, 4, 1, 1,
        1, 1, 3, 5, 1, 4, 3, 5, 5, 1, 0, 4, 3, 1, 3, 1, 5, 1, 4, 1, 5, 0, 0, 1,
        5, 1, 1, 5, 4, 1, 5, 4, 4, 2, 2, 5, 1, 5, 3, 1])
---------------o0o----------------
Predicted:  tensor([

        4, 5, 5, 1, 1, 1, 1, 5, 1, 3, 0, 5, 1, 0, 4, 1])
---------------o0o----------------
Predicted:  tensor([2, 3, 1, 5, 1, 1, 5, 5, 1, 3, 2, 5, 1, 5, 0, 1, 5, 5, 5, 0, 3, 5, 5, 5,
        1, 3, 1, 5, 5, 5, 1, 1, 1, 1, 1, 5, 0, 1, 5, 1, 2, 1, 1, 0, 1, 0, 5, 3,
        5, 5, 2, 1, 1, 5, 1, 1, 5, 5, 5, 5, 0, 5, 5, 0])
Labels:  tensor([3, 4, 1, 5, 1, 1, 5, 5, 1, 3, 2, 0, 1, 5, 4, 1, 5, 5, 4, 0, 3, 5, 5, 5,
        1, 2, 1, 5, 0, 0, 1, 1, 1, 1, 1, 4, 4, 1, 5, 1, 2, 1, 1, 5, 1, 0, 5, 2,
        5, 5, 2, 1, 1, 5, 1, 1, 5, 5, 5, 4, 4, 5, 0, 4])
---------------o0o----------------
Predicted:  tensor([1, 5, 1, 5, 1, 1, 3, 1, 2, 3, 1, 5, 1, 5, 1, 5, 5, 0, 1, 1, 1, 5, 5, 5,
        5, 1, 5, 1, 5, 5, 1, 2, 5, 5, 0, 5, 5, 1, 1, 0, 5, 1, 5, 5, 5, 1, 5, 1,
        1, 5, 1, 5, 5, 5, 0, 5, 5, 0, 1, 1, 5, 1, 5, 5])
Labels:  tensor([1, 5, 1, 5, 1, 1, 3, 1, 2, 2, 1, 4, 1, 5, 1, 5, 5, 4, 1, 1, 1, 4, 5, 0,
        5, 1, 5, 1, 5, 4, 1, 2, 4, 5, 0, 5, 4, 1, 1, 0, 5, 1, 5, 5, 5, 1, 5, 1,
        1, 5, 1, 5, 

        2, 5, 5, 0, 0, 5, 5, 5, 1, 1, 5, 1, 5, 1, 5, 4])
---------------o0o----------------
Predicted:  tensor([5, 5, 5, 1, 5, 5, 1, 1, 1, 1, 1, 0, 1, 1, 5, 1, 1, 5, 0, 1])
Labels:  tensor([5, 5, 5, 1, 0, 5, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 5, 4, 1])
---------------o0o----------------
Test Accuracy on the test set: 83.1511839708561 %
